In [1]:
# Uninstall fastai for solving dependence problems
!pip uninstall fastai -y
# Install packages without internet
!pip install ../input/packages/torch-1.7.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/packages/torchvision-0.8.2-cp37-cp37m-manylinux1_x86_64.whl

Found existing installation: fastai 2.0.19
Uninstalling fastai-2.0.19:
  Successfully uninstalled fastai-2.0.19
Processing /kaggle/input/packages/torch-1.7.1-cp37-cp37m-manylinux1_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.6.0
    Uninstalling torch-1.6.0:
      Successfully uninstalled torch-1.6.0
Processing /kaggle/input/packages/torchvision-0.8.2-cp37-cp37m-manylinux1_x86_64.whl
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.7.0
    Uninstalling torchvision-0.7.0:
      Successfully uninstalled torchvision-0.7.0


In [2]:
import sys
sys.path.append('../input/repvggmodels/')

from repvgg import RepVGG, create_RepVGG_B3g4, create_RepVGG_B3, repvgg_model_convert

In [3]:
import os
import cv2
import time
import random
import joblib
import sklearn
import warnings
import multiprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from skimage import io
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GroupKFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss
from IPython.display import display
from catalyst.data.sampler import BalanceClassSampler

import torch
import torch.nn.functional as F
import torchvision
from torch import nn
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.nn.modules.loss import _WeightedLoss
from torchvision import models
from torchvision import transforms

from albumentations.pytorch import ToTensor, ToTensorV2
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout,
    ShiftScaleRotate, CenterCrop, Resize)

pd.set_option("max_rows", 100)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [5]:
CFG = {
    'seed'       : 42,
    'fold'       : 0 if len(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))==1 else 99,
    'tta'        : 1 if len(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))==1 else 4,
    'img_size'   : 400,
    'valid_bs'   : 32,
    'num_workers': multiprocessing.cpu_count(),
    'device'     : "cuda:0" if torch.cuda.is_available() else "cpu"}

In [6]:
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

# Make a model with noise and clean models

In [7]:
class CassavaDataset(Dataset):
    def __init__(self, df, data_root, 
                 transforms=None, 
                 output_label=True):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms  = transforms
        self.data_root   = data_root
        self.output_label  = output_label
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        img = get_img(f"{self.data_root}/{self.df.loc[index]['image_id']}")
        if self.transforms:
            img = self.transforms(image=img)['image']
        return img

In [8]:
def get_inference_transforms():
    return Compose([
            RandomResizedCrop(CFG['img_size'], CFG['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [9]:
class FFN(nn.Module):
    def __init__(self, backbone=None):
        super(FFN, self).__init__()
        self.backbone = backbone
        self.lr1      = nn.Linear(1000, 256)
        self.relu     = nn.ReLU()
        self.dropout  = nn.Dropout(0.5)
        self.lr2      = nn.Linear(256, 5)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.relu(self.lr1(x))
        x = self.dropout(x)
        x = self.lr2(x)
        return x

In [10]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        image_preds = model(imgs)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [11]:
def predict(df, data_path, model_path, backbone):
    results = np.zeros((len(os.listdir(data_path)), 5))
    models  = [m for m in os.listdir(model_path) if m.find("csv")==-1]
    for model_file in models:
        device  = torch.device(CFG['device'])
        dataset = CassavaDataset(df, data_path, transforms=get_inference_transforms(), output_label=False)

        data_loader = torch.utils.data.DataLoader(
            dataset, 
            batch_size =CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False)

        model = FFN(backbone)
        if CFG['device'] == "cpu":
            model.load_state_dict(torch.load(f"{model_path}{model_file}", map_location="cpu"))
        else:
            model.load_state_dict(torch.load(f"{model_path}{model_file}"))
        backbone.to(device)
        model.to(device)

        preds = []
        with torch.no_grad():
            for _ in range(CFG['tta']):
                preds += [(1/CFG['tta'])*inference_one_epoch(model, data_loader, device)]
        preds    = np.sum(preds, 0)
        results += preds
        del model
        torch.cuda.empty_cache()
        
        if CFG['fold'] == 0:
            return results
        
    return results / len(models)

In [12]:
train      = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train_path = '../input/cassava-leaf-disease-classification/train_images/'
model_path = '../input/cassava-models-trained-with-noise-labels/'
backbone   = create_RepVGG_B3g4(deploy=True)

results_from_noise_model = predict(train, train_path, model_path, backbone)

100%|██████████| 669/669 [06:26<00:00,  1.73it/s]


In [13]:
model_path = '../input/cassava-noised-label-data/'
backbone   = create_RepVGG_B3g4(deploy=True)

results_from_clean_model = predict(train, train_path, model_path, backbone)

100%|██████████| 669/669 [05:49<00:00,  1.91it/s]


In [14]:
df_noise = pd.DataFrame(results_from_noise_model, columns=[f"l{l}_noise" for l in range(5)])
df_clean = pd.DataFrame(results_from_clean_model, columns=[f"l{l}_clean" for l in range(5)])
df_all_labels = pd.concat([df_noise, df_clean], axis=1)

# Feature engineering
df_all_labels["pred_noise"] = np.argmax(results_from_noise_model, 1)
df_all_labels["pred_clean"] = np.argmax(results_from_clean_model, 1)
df_all_labels["diff_noise_proba"] = np.max(results_from_noise_model, 1) - (results_from_clean_model * np.identity(5)[np.argmax(results_from_noise_model, 1)]).sum(1)
df_all_labels["diff_clean_proba"] = np.max(results_from_clean_model, 1) - (results_from_noise_model * np.identity(5)[np.argmax(results_from_clean_model, 1)]).sum(1)
df_all_labels["label"] = train.label

print(df_all_labels.shape)
df_all_labels.head(5)

(21397, 15)


,l0_noise,l1_noise,l2_noise,l3_noise,l4_noise,l0_clean,l1_clean,l2_clean,l3_clean,l4_clean,pred_noise,pred_clean,diff_noise_proba,diff_clean_proba,label
0,0.863532,0.041206,0.027994,0.001638,0.065630,1.716995e-01,0.028918,0.754741,0.003274,4.136809e-02,0,2,0.691832,0.726747,0
1,0.004477,0.031961,0.005028,0.953340,0.005195,3.250370e-07,0.000802,0.000023,0.999173,7.283189e-07,3,3,-0.045833,0.045833,3
2,0.008027,0.040088,0.014453,0.044168,0.893264,7.758388e-03,0.039392,0.007170,0.014944,9.307355e-01,4,4,-0.037471,0.037471,1
3,0.000558,0.990621,0.000922,0.001199,0.006700,1.411741e-03,0.977580,0.002137,0.013113,5.758080e-03,1,1,0.013041,-0.013041,1
4,0.000548,0.000283,0.000363,0.997288,0.001518,5.281977e-07,0.000018,0.000008,0.999967,6.404779e-06,3,3,-0.002679,0.002679,3


In [15]:
def show_accuracy(p, l):
    if p.ndim==2:
        print(np.sum(np.argmax(p, 1) == np.array(l)) / l.shape[0])
    else:
        print(np.sum(p == np.array(l)) / l.shape[0])

show_accuracy(results_from_noise_model, train.label)
show_accuracy(results_from_clean_model, train.label)

0.8880216852829836
0.8784876384539889


In [16]:
df_miss = df_all_labels[df_all_labels.pred_noise != df_all_labels.pred_clean]
svm = SVC(kernel='linear', random_state=42)
svm.fit(np.array(df_miss.drop("label", axis=1)), np.array(df_miss.label))

SVC(kernel='linear', random_state=42)

In [17]:
df_all_labels.loc[df_all_labels.pred_noise != df_all_labels.pred_clean, "label"] = svm.predict(df_miss.drop("label", axis=1))
preds_with_svm = df_all_labels.label
show_accuracy(preds_with_svm, train.label)

0.9636864981072113


# Inference

In [18]:
test_path  = '../input/cassava-leaf-disease-classification/test_images/'
test = pd.DataFrame()
test['image_id'] = list(os.listdir(test_path))
model_path = '../input/cassava-models-trained-with-noise-labels/'
backbone   = create_RepVGG_B3g4(deploy=True)

results_from_noise_model = predict(test, test_path, model_path, backbone)

100%|██████████| 1/1 [00:00<00:00,  1.29it/s]


In [19]:
model_path = '../input/cassava-noised-label-data/'
backbone   = create_RepVGG_B3g4(deploy=True)

results_from_clean_model = predict(test, test_path, model_path, backbone)

100%|██████████| 1/1 [00:00<00:00, 10.82it/s]


In [20]:
df_noise = pd.DataFrame(results_from_noise_model, columns=[f"l{l}_noise" for l in range(5)])
df_clean = pd.DataFrame(results_from_clean_model, columns=[f"l{l}_clean" for l in range(5)])
df_all_labels = pd.concat([df_noise, df_clean], axis=1)

# Feature engineering
df_all_labels["pred_noise"] = np.argmax(results_from_noise_model, 1)
df_all_labels["pred_clean"] = np.argmax(results_from_clean_model, 1)
df_all_labels["diff_noise_proba"] = np.max(results_from_noise_model, 1) - (results_from_clean_model * np.identity(5)[np.argmax(results_from_noise_model, 1)]).sum(1)
df_all_labels["diff_clean_proba"] = np.max(results_from_clean_model, 1) - (results_from_noise_model * np.identity(5)[np.argmax(results_from_clean_model, 1)]).sum(1)

print(df_all_labels.shape)
df_all_labels.head(5)

(1, 14)


,l0_noise,l1_noise,l2_noise,l3_noise,l4_noise,l0_clean,l1_clean,l2_clean,l3_clean,l4_clean,pred_noise,pred_clean,diff_noise_proba,diff_clean_proba
0,0.008613,0.010346,0.131948,0.000946,0.848147,0.009265,0.00341,0.098546,0.005546,0.883233,4,4,-0.035086,0.035086


In [21]:
df_miss = df_all_labels[df_all_labels.pred_noise != df_all_labels.pred_clean]
df_all_labels["label"] = df_all_labels.pred_noise
if 0 < df_miss.shape[0]:
    df_all_labels.loc[df_all_labels.pred_noise != df_all_labels.pred_clean, "label"] = svm.predict(df_miss)
    
preds_final = df_all_labels.label

In [22]:
test['label'] = preds_final
test.head()

,image_id,label
0,2216849948.jpg,4


In [23]:
test.to_csv('submission.csv', index=False)